####  Deliverable 2. Create a Customer Travel Destinations Map

Use input statements to retrieve customer weather preferences, then use those preferences to identify potential travel destinations and nearby hotels. Then, show those destinations on a marker layer map with pop-up markers.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.7297,-155.0900,72.28,77,20,8.05,few clouds
1,1,Albany,US,42.6001,-73.9662,69.03,84,98,1.99,overcast clouds
2,2,Provideniya,RU,64.3833,-173.3000,37.38,78,31,0.92,scattered clouds
3,3,Saldanha,ZA,-33.0117,17.9442,62.65,78,0,12.53,clear sky
4,4,Turukhansk,RU,65.8167,87.9833,31.73,65,83,3.89,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 72
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
cities_df.head(10) 

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.7297,-155.0900,72.28,77,20,8.05,few clouds
13,13,Bertoua,CM,4.5833,13.6833,75.47,93,42,2.46,light rain
14,14,Atuona,PF,-9.8000,-139.0333,76.73,77,32,22.84,scattered clouds
17,17,Vaini,TO,-21.2000,-175.2000,73.56,88,75,21.39,broken clouds
18,18,Dingle,PH,10.9995,122.6711,76.68,88,100,2.51,light rain
20,20,Vao,NC,-22.6667,167.4833,73.90,80,0,10.00,clear sky
26,26,Rikitea,PF,-23.1203,-134.9692,72.82,73,0,14.41,clear sky
29,29,Gat,IL,31.6100,34.7642,79.48,100,18,10.40,few clouds
32,32,Kahului,US,20.8947,-156.4700,78.84,70,20,12.66,few clouds
40,40,Arraial Do Cabo,BR,-22.9661,-42.0278,76.86,65,0,10.36,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
cities_df.count()

City_ID                137
City                   137
Country                137
Lat                    137
Lng                    137
Max Temp               137
Humidity               137
Cloudiness             137
Wind Speed             137
Current Description    137
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,72.28,few clouds,19.7297,-155.0900,
13,Bertoua,CM,75.47,light rain,4.5833,13.6833,
14,Atuona,PF,76.73,scattered clouds,-9.8000,-139.0333,
17,Vaini,TO,73.56,broken clouds,-21.2000,-175.2000,
18,Dingle,PH,76.68,light rain,10.9995,122.6711,
...,...,...,...,...,...,...,...
559,Hengyang,CN,74.48,overcast clouds,26.8881,112.6150,
560,Bandarbeyla,SO,78.62,clear sky,9.4942,50.8122,
563,Dawlatabad,AF,77.41,clear sky,36.4141,64.9053,
566,Port Moresby,PG,76.42,few clouds,-9.4431,147.1797,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] !='']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,Bertoua,CM,75.47,light rain,4.5833,13.6833,Hotel De Paris
26,Rikitea,PF,72.82,clear sky,-23.1203,-134.9692,People ThankYou
57,Puerto Ayora,EC,73.36,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
97,Nzerekore,GN,77.31,light rain,7.7562,-8.8179,Hotel de la Gare & Germanho Club
100,Ahuimanu,US,78.51,few clouds,21.4447,-157.8378,Paradise Bay Resort
124,Inhambane,MZ,74.26,clear sky,-23.8650,35.3833,Oceano
136,Bengkulu,ID,75.31,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
154,Kloulklubed,PW,75.70,broken clouds,7.0419,134.2556,Storyboard Beach Resort
155,Show Low,US,73.65,clear sky,34.2542,-110.0298,Hampton Inn & Suites Show Low-Pinetop
169,Pangai,TO,75.81,light rain,-19.8000,-174.3500,Ha'apai Beach Resort


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
vacation_df = pd.read_csv('WeatherPy_Vacation.csv')

vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,13,Bertoua,CM,75.47,light rain,4.5833,13.6833,Hotel De Paris
1,26,Rikitea,PF,72.82,clear sky,-23.1203,-134.9692,People ThankYou
2,57,Puerto Ayora,EC,73.36,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
3,97,Nzerekore,GN,77.31,light rain,7.7562,-8.8179,Hotel de la Gare & Germanho Club
4,100,Ahuimanu,US,78.51,few clouds,21.4447,-157.8378,Paradise Bay Resort


In [12]:
# Using the template add the hotel marks to the heatmap

info_box_template = """
<d1>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>

"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store it in a new DataFrame
locations = vacation_df[['Lat', 'Lng']]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))